Since my pc couldn't connect to wiki directly, I downloaded the html page through a VPN and save it locally. Pandas read the table from this locally saved html page.

In [4]:
import pandas as pd
url='file:///C:/Users/glp_a/OneDrive/Confidential/IBM%20DATA%20SCIENCE/Capstone/List%20of%20postal%20codes%20of%20Canada_%20M%20-%20Wikipedia.html'
df=pd.read_html(url, header=0)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M9Z,Not assigned,Not assigned
1,M9Y,Not assigned,Not assigned
2,M9X,Not assigned,Not assigned
3,M9W,Etobicoke,"Northwest, West Humber - Clairville"
4,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


Check size of original table

In [17]:
df.shape

(180, 3)

Drop lines with a borough that is Not assigned, then check size.

In [5]:
df.drop(df[df['Borough']=='Not assigned'].index,axis=0,inplace=True)
df.shape

(103, 3)

In [6]:
df.head()

,Postal Code,Borough,Neighbourhood
3,M9W,Etobicoke,"Northwest, West Humber - Clairville"
4,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
7,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
8,M9P,Etobicoke,Westmount
9,M9N,York,Weston


Merge different neighbourhoods with the same postal code into one row, seprated by ','.

In [7]:
df1 = df.groupby(['Postal Code','Borough'])['Neighbourhood'].agg(','.join).reset_index()
df1.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df1.shape

(103, 3)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
neiglist=[]
for bor,neig in zip(df1['Borough'],df1['Neighbourhood']):
    if neig == 'Not assigned':
        neiglist.append(bor)
    else:
        neiglist.append(neig) 
df1['Neighbourhood']=neiglist   

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [10]:
print('Now there\'s',df1.shape[0],'rows in the dataframe' )

Now there's 103 rows in the dataframe


Since I'm not able to get the geographical coordinates of the neighborhoods using the Geocoder package, I use the csv file instead.

In [11]:
podf=pd.read_csv('Geospatial_Coordinates.csv')
podf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the 2 tables into one

In [13]:
df2=pd.merge(df1,podf,on='Postal Code',how='left')
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
